# Run illumination correction on data

Note: We load in the CellProfiler IC pipeline to use for this process.

## Import libraries

In [1]:
import pathlib
import pprint

import sys

sys.path.append("../utils")
import cp_parallel

## Set paths and variables

### Set the constants

In [2]:
# set the run type for the parallelization
run_name = "illum_correction"

# batch to process
batch = "batch_1"

### Set up paths

In [3]:
# set base directory for where the images are located
base_dir = pathlib.Path(
    f"/media/18tbdrive/CFReT_screening_data/compound_screen/{batch}"
).resolve(strict=True)

# list for plate names
plate_names = []

# iterate through each platemap folder
plate_names = [p.name for p in base_dir.rglob("platemap_*/**/CARD*") if p.is_dir()]

print("There are a total of", len(plate_names), "plates. The names of the plates are:")
for plate in plate_names:
    print(plate)

There are a total of 16 plates. The names of the plates are:
CARD-CelIns-CX7_251203170001
CARD-CelIns-CX7_251205100001
CARD-CelIns-CX7_251208160001
CARD-CelIns-CX7_251210180001
CARD-CelIns-CX7_251023210001
CARD-CelIns-CX7_251125110001
CARD-CelIns-CX7_251124150001
CARD-CelIns-CX7_251126130001
CARD-CelIns-CX7_251212100001
CARD-CelIns-CX7_251213150001
CARD-CelIns-CX7_251211180001
CARD-CelIns-CX7_251212180001
CARD-CelIns-CX7_251130110002
CARD-CelIns-CX7_251202100001
CARD-CelIns-CX7_251201110001
CARD-CelIns-CX7_251201190001


## Create dictionary with all plate data to run CellProfiler in parallel

In [4]:
# set path to the illum pipeline
path_to_pipeline = pathlib.Path("./pipeline/illum.cppipe").resolve(strict=True)

# set main output dir for all plates
output_base_dir = pathlib.Path("./Corrected_Images").resolve(strict=False)
output_base_dir.mkdir(exist_ok=True)
output_dir = pathlib.Path(f"{output_base_dir}/{batch}").resolve(strict=False)
output_dir.mkdir(exist_ok=True)

# create plate info dictionary
plate_info_dictionary = {}

# recursively find all CARD* folders under platemap_*
for plate_folder in base_dir.rglob("platemap_*/**/CARD*"):
    if plate_folder.is_dir():
        # determine the parent platemap folder name
        platemap_folder_name = plate_folder.parents[
            1
        ].name  # 1 level up from plate_folder inside rglob pattern

        # create nested output dir: Corrected_Images/platemap_#/plate
        plate_output_dir = output_dir / platemap_folder_name / plate_folder.name

        # only create output dir if it doesn't exist or is empty
        if not plate_output_dir.exists() or not any(plate_output_dir.iterdir()):
            plate_output_dir.mkdir(parents=True, exist_ok=True)

            # add info to dictionary
            plate_info_dictionary[plate_folder.name] = {
                "path_to_images": plate_folder.resolve(strict=True),
                "path_to_output": plate_output_dir.resolve(strict=True),
                "path_to_pipeline": path_to_pipeline,
            }
        else:
            print(
                f"{plate_output_dir} already exists and contains files, skipping creation and dictionary."
            )

# view the dictionary
pprint.pprint(plate_info_dictionary, indent=4)

{   'CARD-CelIns-CX7_251023210001': {   'path_to_images': PosixPath('/media/18tbdrive/CFReT_screening_data/compound_screen/batch_1/platemap_1/1a/CARD-CelIns-CX7_251023210001'),
                                        'path_to_output': PosixPath('/home/jenna/targeted_fibrosis_drug_screen/1.illumination_correction/Corrected_Images/batch_1/platemap_1/CARD-CelIns-CX7_251023210001'),
                                        'path_to_pipeline': PosixPath('/home/jenna/targeted_fibrosis_drug_screen/1.illumination_correction/pipeline/illum.cppipe')},
    'CARD-CelIns-CX7_251124150001': {   'path_to_images': PosixPath('/media/18tbdrive/CFReT_screening_data/compound_screen/batch_1/platemap_1/1b/CARD-CelIns-CX7_251124150001'),
                                        'path_to_output': PosixPath('/home/jenna/targeted_fibrosis_drug_screen/1.illumination_correction/Corrected_Images/batch_1/platemap_1/CARD-CelIns-CX7_251124150001'),
                                        'path_to_pipeline': PosixPath('

## Run CellProfiler Parallel

Note: We do not run this code cell as we will run this process through the script.

In [ ]:
# if dictionary is not empty, run CellProfiler in parallel
if plate_info_dictionary:
    cp_parallel.run_cellprofiler_parallel(
        plate_info_dictionary=plate_info_dictionary,
        run_name=run_name,
        group_level="plate",
    )
else:
    print("No new plates to process. Exiting script.")